# Exploratory Analysis of a Monthly Dataset

In [ ]:
# do the below to load the python utilities from py_depends

In [ ]:
cd ..

In [ ]:
import sys
from py_depends.utils import *
from aux.dtypes import *

# visualizations and such
import seaborn
import qgrid

# sanity check - assert the import worked correctly
try:
    assert 'flight-delays' in os.getcwd()
except AssertionError:
    print('Doesn\'t look like the cd .. hack worked - restart the kernel and try again')

_First, some housekeeping. Reading in a sample file,_

In [ ]:
%time jan_2018_df = unzipFile('./sample_data/On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2018_1.zip')

%time subs_jan_2018_df = subsetRenameRetype(jan_2018_df, new_cols)

%time subs_jan_2018_df = shapeDatetime(subs_jan_2018_df, ['crs_dep_time', 'dep_time','crs_arr_time', 'arr_time', 'first_dep_time', 'diverted_wheels_off_1', 'diverted_wheels_off_2', 'diverted_wheels_off_3'], 'flight_date')

## Descriptive stats

### Airports

In [ ]:
qgrid.show_grid(subs_jan_2018_df.dest.value_counts())

In [ ]:
pd.to_numeric(jan_2018_df['CRSDepTime'], errors='coerce').fillna(0)

In [ ]:
jan_2018_df['CRSDepTime']